In [1]:
import numpy as np
import tensorflow as tf

In [2]:
import Focusing

In [3]:
import ReadWriteVectorGeneration

PLEASE NOTE TO IMPLEMENT INTIAL STATES AS tf.float64 TYPE

In [4]:
tf.version.VERSION

'2.0.0'

In [114]:
class NTMCell(tf.keras.layers.AbstractRNNCell):
    
    
    '''
        ARGUMENTS TO THE call FUNCTION::--
        
        ////////////////////          TODO       /////////////////////////          
        
        Few Points:
        So the combination of inputs and preious_states will be used to calculate w_t, this w_t will be used for READING, the resulting vector r_t will be outputted.
        The same w_t is also used for Writing Purposes, which results in a updated Memory Matrix, which is also returned as the next_state's element.
        
        
        ////////////////////// TODO  //////////////////////////
        
        
        MORE ELABORATION ON THE METHODOLOGY NEEDS TO BE DONE 
        
        
    '''
    
    #the Initialiser function
    def __init__(self, rnn_size, memory_rows, memory_columns, num_read_heads, num_write_heads, num_bits_per_output_vector, addressing_type = 'LOC', shift_range = tf.range(-1,2), controller = tf.keras.layers.LSTMCell 
                     ): 
        #output_dim: Number of bits per output vector
        
        super().__init__(**kwargs)
        
        
        self.rnn_size = rnn_size
        self.memory_rows = memory_rows       #The "N" or "size of memory" in Literature
        self.memory_columns = memory_columns      #The "M" or "memory vector's dimension" in Literature
        self.num_read_heads = num_read_heads
        self.num_write_heads = num_write_heads
        self.num_bits_per_output_vector = num_bits_per_output_vector
        self.addressing_type = addressing_type
        
        if ((self.addressing_type != 'LOC') or (self.addressing_type != 'CONT')):
            raise ValueError('Incorrect Addressing Type: Allowed values are "LOC" for Location based Focusing and "CONT" for Content based Focusing.')

        
        self.shift_range = shift_range
        
            
            
        #self.state_size TODO:::: Define a @property for this which returns the dictionary containing the states as discussed.
        
        
        
        self.output_size = self.num_bits_per_output_vector
        self.parameter_count = self.memory_columns + 1 + 1 + len(self.shift_range) + 1 + self.memory_columns + self.memory_columns   #As Discussed
            
        
        self.controller = controller(self.rnn_size)     #The controller to use
        
        
        
        self.total_num_heads = self.num_read_heads + self.num_write_heads
        
        self.controller_to_parameter = tf.keras.layers.Dense(units = self.parameter_count,use_bias=True)
        
        self.parameter_matrix_to_NTM_output = tf.keras.layers.Dense(units = self.output_size,use_bias=True)
        
        
        
    #The call function which will be called at each time/input step
    def call(self, inputs, previous_state):
    
        prev_read_vectors = previous_state['current_read_vectors']
        
        prev_controller_state = previous_state['current_controller_state']
        
        input_to_controller = tf.concat([inputs] + prev_read_vectors, axis = 1)
        
        controller_output, controller_state = self.controller(input_to_controller, prev_controller_state)
        
        parameter_matrix = self.controller_to_parameter(controller_output)
        
        
        M_prev = previous_state['Memory_Matrix']
        
        w_prev_list = previous_state['current_weights_list']
        
        
        # Creating a list of all Parameters Emitted by the Heads
        controller_parameter_list = tf.split(parameter_matrix[0],[self.memory_columns ,1, 1, len(self.shift_range), 1, self.memory_columns, self.memory_columns], axis = 1)
        # Applying Layers on the parameter Matrix as per the Paramter's Location.
        
        
        #Useless Shape check
        assert self.total_num_heads == controller_parameter_list[0].shape[0]
        
        
        #For k_t:-
        controller_parameter_list[0] = tf.tanh(controller_parameter_list[0])
        k_t = controller_parameter_list[0]
        #For beta_t:-
        controller_parameter_list[1] = tf.sigmoid(controller_parameter_list[1]) * 5 #Attenuating Factor
        beta_t = controller_parameter_list[1]
        #For g_t:-
        controller_parameter_list[2] = tf.sigmoid(controller_parameter_list[2])
        g_t = controller_parameter_list[2]
        #For s_t:-
        controller_parameter_list[3] = tf.nn.softmax(controller_parameter_list[3], axis = 1)
        s_t = controller_parameter_list[3]
        #The above s_t is one of the points where we can improve
        #For gamma_t:-
        controller_parameter_list[4] = tf.math.log(controller_parameter_list[4]) + 1 #Since gamma >= 1
        gamma_t = controller_parameter_list[4]
        #For a_t:-
        controller_parameter_list[5] = tf.tanh(controller_parameter_list[5])
        a_t = controller_parameter_list[5]
        #For e_t:-
        controller_parameter_list[6] = tf.sigmoid(controller_parameter_list[6])
        e_t = controller_parameter_list[6]
        
        
        
        #Finally, we find the w_t at this step.
        
        All_Heads_W = []
        
        if self.addressing_type == 'LOC':
            for i in range(self.total_num_heads):
                w_t = Focusing.LocationFocusing(k_t[i], M_prev, beta_t[i], g_t[i], w_prev_list[i],s_t[i], gamma_t[i]) #Note that Cosine Similarity is used
                All_Heads_W.append(w_t)
            
        elif self.addressing_type == 'CONT':
            for i in range(self.total_num_heads):
                w_t = Focusing.ContentFocusing(k_t[i], M_prev, beta_t[i]) #Note that Cosine Similarity is used
                All_Heads_W.append(w_t)
                
        #It would be better if we find a vectorized way to go about it.
        
        #Now for Writing and Reading
        #NOTE: We consider first self.num_read_heads as READ HEADS and the rest as WRITE HEADS
        
        read_Heads_w = All_Heads_W[:self.num_read_heads]
        write_Heads_w = All_Heads_W[self.num_read_heads:]
        
        assert len(write_Heads_w) == self.num_write_heads
        
        r_t_list = []
        for read_head_w in read_Heads_w:
            r_t = ReadWriteVectorGeneration.ReadVector(M_prev,read_head_w)
            r_t_list.append(r_t)
            
        for i, write_head_w in enumerate(write_Heads_w):
            M_prev = ReadWriteVectorGeneration.WriteOnMemory(M_prev, write_head_w, e_t[self.num_read_heads + i], a_t[self.num_read_heads + i])
           #Note that the M_prev is now the updated Memory Matrix by action from all the WRITE HEADS
        
        assert len(r_t_list) == len(prev_read_vectors)
        
        
        state = {
                     'current_read_vectors' : r_t_list,
                     'current_controller_state' : controller_state,
                     'current_weights_list' : All_Heads_W,
                     'Memory_Matrix' : M_prev
                }
        
        
        NTM_output = self.parameter_matrix_to_NTM_output(parameter_matrix)
        
        
        
        return NTM_output ,state
    
    def 

# Rough Testing Below

## Testing Final Weights Generation

In [24]:
N = 15
M = 8
HEADS = 40

In [25]:
g_t = tf.random.uniform((HEADS,1))
w_prev = tf.random.uniform((HEADS,N))
s_t = tf.nn.softmax(tf.random.uniform((HEADS,3)),axis = 1)
gamma_t = tf.random.uniform((HEADS,1))*2 + 1 

In [26]:
k_t = tf.random.uniform((HEADS,M))
b_t = tf.random.uniform((HEADS,1))*2 + 1 
M_t = tf.random.uniform((N,M))

In [27]:
All_Heads_W = []
for i in range(HEADS):
    w_t = Focusing.LocationFocusing(k_t[i], M_t, b_t[i], g_t[i], w_prev[i],s_t[i], gamma_t[i]) #Note that Cosine Similarity is used
    All_Heads_W.append(w_t)

In [28]:
i = 0
Focusing.LocationFocusing(k_t[i], M_t, b_t[i], g_t[i], w_prev[i] ,s_t[i], gamma_t[i])

<tf.Tensor: id=106378, shape=(15,), dtype=float32, numpy=
array([0.06764598, 0.0660094 , 0.06634461, 0.06764598, 0.0660094 ,
       0.06634461, 0.06764598, 0.0660094 , 0.06634461, 0.06764598,
       0.0660094 , 0.06634461, 0.06764598, 0.0660094 , 0.06634461],
      dtype=float32)>

In [29]:
gamma_t[0]

<tf.Tensor: id=106382, shape=(1,), dtype=float32, numpy=array([2.7387912], dtype=float32)>

In [30]:
All_Heads_W

[<tf.Tensor: id=2698, shape=(15,), dtype=float32, numpy=
 array([0.06764598, 0.0660094 , 0.06634461, 0.06764598, 0.0660094 ,
        0.06634461, 0.06764598, 0.0660094 , 0.06634461, 0.06764598,
        0.0660094 , 0.06634461, 0.06764598, 0.0660094 , 0.06634461],
       dtype=float32)>, <tf.Tensor: id=5290, shape=(15,), dtype=float32, numpy=
 array([0.06890429, 0.07049696, 0.06059873, 0.06890429, 0.07049696,
        0.06059873, 0.06890429, 0.07049696, 0.06059873, 0.06890429,
        0.07049696, 0.06059873, 0.06890429, 0.07049696, 0.06059873],
       dtype=float32)>, <tf.Tensor: id=7882, shape=(15,), dtype=float32, numpy=
 array([0.06615897, 0.06245341, 0.07138763, 0.06615897, 0.06245341,
        0.07138763, 0.06615897, 0.06245341, 0.07138763, 0.06615897,
        0.06245341, 0.07138763, 0.06615897, 0.06245341, 0.07138763],
       dtype=float32)>, <tf.Tensor: id=10474, shape=(15,), dtype=float32, numpy=
 array([0.0679554 , 0.06770483, 0.06433976, 0.0679554 , 0.06770483,
        0.06433976,

In [31]:
a = np.array([1,2,3,4])

In [32]:
b = np.array([0.1,0.8,0.1])

In [33]:
np.convolve(a,b,'same')

array([1. , 2. , 3. , 3.5])

In [34]:
a

array([1, 2, 3, 4])

## Testing Read and Write Vectors

In [35]:
w_prev = tf.nn.softmax(w_prev, axis = 1)

In [36]:
ReadWriteVectorGeneration.ReadVector(M_t,w_prev[1])

<tf.Tensor: id=106392, shape=(8,), dtype=float32, numpy=
array([0.44150448, 0.53284335, 0.5331958 , 0.4082237 , 0.35997802,
       0.49569654, 0.49706772, 0.44460157], dtype=float32)>

In [37]:
e_t = tf.sigmoid(tf.random.uniform((HEADS,M)))
a_t = tf.sigmoid(tf.random.uniform((HEADS,M)))

In [38]:
write_Heads_w = All_Heads_W[20:]

In [39]:
for i, write_head_w in enumerate(write_Heads_w):
    M_t = ReadWriteVectorGeneration.WriteOnMemory(M_t, write_head_w, e_t[20 + i], a_t[20 + i])
    

In [40]:
%timeit dict(a=2,cat=5)

209 ns ± 42.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [41]:
%timeit {'a':2, 'cat':5}

67.5 ns ± 0.449 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [51]:
a = tf.random.uniform((M,))
a_list = []
for i in range(40):
    a_list.append(tf.random.uniform((M,)))

In [57]:
a_list

[<tf.Tensor: id=107545, shape=(8,), dtype=float32, numpy=
 array([0.7958343 , 0.2405392 , 0.5578928 , 0.5485002 , 0.87103474,
        0.62820005, 0.01486003, 0.44726622], dtype=float32)>,
 <tf.Tensor: id=107552, shape=(8,), dtype=float32, numpy=
 array([0.18382478, 0.31391227, 0.32524955, 0.9693028 , 0.87679935,
        0.5045861 , 0.80644333, 0.6368617 ], dtype=float32)>,
 <tf.Tensor: id=107559, shape=(8,), dtype=float32, numpy=
 array([0.65347934, 0.18584335, 0.5184344 , 0.4089533 , 0.8763708 ,
        0.933172  , 0.69516075, 0.7479063 ], dtype=float32)>,
 <tf.Tensor: id=107566, shape=(8,), dtype=float32, numpy=
 array([0.32188535, 0.48913968, 0.9399965 , 0.49112034, 0.6137265 ,
        0.1685077 , 0.75082386, 0.02874684], dtype=float32)>,
 <tf.Tensor: id=107573, shape=(8,), dtype=float32, numpy=
 array([0.67042744, 0.5801518 , 0.6172439 , 0.52459276, 0.4075538 ,
        0.29342484, 0.5674629 , 0.9692135 ], dtype=float32)>,
 <tf.Tensor: id=107580, shape=(8,), dtype=float32, numpy=
 a

In [68]:
tf.keras.layers.AbstractRNNCell.add_weight?

Signature:
tf.keras.layers.AbstractRNNCell.add_weight(
    self,
    name=None,
    shape=None,
    dtype=None,
    initializer=None,
    regularizer=None,
    trainable=None,
    constraint=None,
    partitioner=None,
    use_resource=None,
    synchronization=<VariableSynchronization.AUTO: 0>,
    aggregation=<VariableAggregation.NONE: 0>,
    **kwargs,
)
Docstring:
Adds a new variable to the layer.

Arguments:
  name: Variable name.
  shape: Variable shape. Defaults to scalar if unspecified.
  dtype: The type of the variable. Defaults to `self.dtype` or `float32`.
  initializer: Initializer instance (callable).
  regularizer: Regularizer instance (callable).
  trainable: Boolean, whether the variable should be part of the layer's
    "trainable_variables" (e.g. variables, biases)
    or "non_trainable_variables" (e.g. BatchNorm mean and variance).
    Note that `trainable` cannot be `True` if `synchronization`
    is set to `ON_READ`.
  constraint: Constraint instance (callable).
  

## Controller Testing

In [54]:
controller = tf.keras.layers.LSTMCell(100)

In [69]:
read_vector_list = tf.random.uniform((HEADS,M))

In [88]:
controller_output = tf.keras.layers.Dense(8,use_bias=True)(tf.concat([a_list] + read_vector_list,axis = 1))

In [79]:
read_vector_list[0][0] + a_list[0][0]

<tf.Tensor: id=107873, shape=(), dtype=float32, numpy=1.378221>

In [89]:
controller_output.shape

TensorShape([1, 40, 8])

In [109]:
pm = tf.keras.layers.Dense(3*M + 3 + 3)(controller_output)

In [112]:
tf.keras.layers.Dense(6)(pm[0])

<tf.Tensor: id=108419, shape=(40, 6), dtype=float32, numpy=
array([[ 2.42249340e-01,  4.28915918e-01,  5.22147715e-01,
        -1.10394299e+00,  1.17238319e+00, -7.26243258e-01],
       [ 7.43943095e-01,  6.48761690e-01, -4.80765283e-01,
        -1.05578780e+00, -9.06221420e-02, -1.30426693e+00],
       [ 7.61377871e-01,  2.09608167e-01, -2.86324322e-01,
        -9.59889054e-01,  6.82249129e-01, -9.17188346e-01],
       [ 3.41309160e-02,  4.34828788e-01,  4.65557873e-01,
        -7.48184681e-01,  6.64874494e-01, -6.43898547e-01],
       [ 2.01101378e-01,  2.95013607e-01,  6.49619699e-01,
        -5.43551862e-01,  1.16356039e+00, -4.88495171e-01],
       [ 4.66551691e-01,  8.56188685e-02,  3.86032090e-02,
        -2.42460132e-01,  2.80426443e-01, -4.97029901e-01],
       [ 4.49807823e-01,  4.28324759e-01,  4.98643637e-01,
        -6.91228986e-01,  1.01384139e+00, -6.82130098e-01],
       [ 8.73904824e-01,  7.17743874e-01, -3.24182093e-01,
        -1.52445436e+00,  8.13355803e-01, -1.347

In [119]:
tf.concat([tf.expand_dims(read_vector_list, 0) for _ in range(24)], axis=0)

<tf.Tensor: id=108475, shape=(24, 40, 8), dtype=float32, numpy=
array([[[0.58238673, 0.6871129 , 0.998054  , ..., 0.35948265,
         0.49559212, 0.4939419 ],
        [0.29837275, 0.7203516 , 0.6919899 , ..., 0.667313  ,
         0.9397123 , 0.04958665],
        [0.83356595, 0.4872613 , 0.06348872, ..., 0.45554817,
         0.16243148, 0.8563956 ],
        ...,
        [0.6215521 , 0.34550548, 0.7988658 , ..., 0.23126376,
         0.8698598 , 0.34368753],
        [0.29138434, 0.3371259 , 0.7330675 , ..., 0.8262805 ,
         0.9214444 , 0.55424345],
        [0.07794499, 0.5211084 , 0.72069216, ..., 0.05690455,
         0.19068575, 0.58132994]],

       [[0.58238673, 0.6871129 , 0.998054  , ..., 0.35948265,
         0.49559212, 0.4939419 ],
        [0.29837275, 0.7203516 , 0.6919899 , ..., 0.667313  ,
         0.9397123 , 0.04958665],
        [0.83356595, 0.4872613 , 0.06348872, ..., 0.45554817,
         0.16243148, 0.8563956 ],
        ...,
        [0.6215521 , 0.34550548, 0.7988658 ,

In [124]:
ta = tf.TensorArray(dtype=tf.float32, size=20)

In [130]:
ta.unstack(tf.transpose?

Signature: tf.transpose(a, perm=None, conjugate=False, name='transpose')
Docstring:
Transposes `a`.

Permutes the dimensions according to `perm`.

The returned tensor's dimension i will correspond to the input dimension
`perm[i]`. If `perm` is not given, it is set to (n-1...0), where n is
the rank of the input tensor. Hence by default, this operation performs a
regular matrix transpose on 2-D input Tensors. If conjugate is True and
`a.dtype` is either `complex64` or `complex128` then the values of `a`
are conjugated and transposed.

@compatibility(numpy)
In `numpy` transposes are memory-efficient constant time operations as they
simply return a new view of the same data with adjusted `strides`.

TensorFlow does not support strides, so `transpose` returns a new tensor with
the items permuted.
@end_compatibility

For example:

```python
x = tf.constant([[1, 2, 3], [4, 5, 6]])
tf.transpose(x)  # [[1, 4]
                 #  [2, 5]
                 #  [3, 6]]

# Equivalently
tf.transpose(x,

In [ ]:
ta.unstack(tf.transpose

In [143]:
a = np.array([1,2,3,4,5,6,0])

In [135]:
b = tf.random.uniform((HEADS,M))

In [146]:
tf.concat([1] + b, axis = 1)

<tf.Tensor: id=108493, shape=(40, 8), dtype=float32, numpy=
array([[1.3934572, 1.8415847, 1.5576944, 1.1662494, 1.8862607, 1.3659568,
        1.0904926, 1.7833703],
       [1.0211182, 1.6074501, 1.8878678, 1.6938491, 1.474513 , 1.6293229,
        1.3422519, 1.4611118],
       [1.0679487, 1.4180768, 1.5142341, 1.9332721, 1.0428125, 1.1839468,
        1.9850836, 1.905674 ],
       [1.512944 , 1.4897838, 1.1008905, 1.5514795, 1.2069813, 1.4097531,
        1.2336558, 1.4328662],
       [1.7728443, 1.8993512, 1.3469207, 1.6982868, 1.7703989, 1.6601863,
        1.3111985, 1.68536  ],
       [1.4795976, 1.492899 , 1.3387004, 1.2239287, 1.8639324, 1.1606152,
        1.3761445, 1.3843462],
       [1.970603 , 1.4159588, 1.6090815, 1.4226836, 1.6520628, 1.2641418,
        1.9727721, 1.2493689],
       [1.6856592, 1.8549848, 1.0699368, 1.9516469, 1.3968012, 1.7385411,
        1.822167 , 1.4389442],
       [1.8494685, 1.6810184, 1.4817904, 1.7523859, 1.4897743, 1.6572026,
        1.669008 , 1.79562

In [148]:
def _expand(x, dim, N):
    return tf.concat([tf.expand_dims(x, dim) for _ in range(N)], axis=dim)


In [168]:
r_vector_list = [_expand(tf.nn.tanh(init_r[i]), dim=0, N=39) for i in range(20)]

In [164]:
init_r = [tf.random.uniform((M,)) for _ in range(20)]

In [169]:
len(r_vector_list)

20

In [170]:
r_vector_list

[<tf.Tensor: id=114962, shape=(39, 8), dtype=float32, numpy=
 array([[0.3643731 , 0.5444552 , 0.34602585, 0.37885317, 0.5664864 ,
         0.68334967, 0.6807079 , 0.6617872 ],
        [0.3643731 , 0.5444552 , 0.34602585, 0.37885317, 0.5664864 ,
         0.68334967, 0.6807079 , 0.6617872 ],
        [0.3643731 , 0.5444552 , 0.34602585, 0.37885317, 0.5664864 ,
         0.68334967, 0.6807079 , 0.6617872 ],
        [0.3643731 , 0.5444552 , 0.34602585, 0.37885317, 0.5664864 ,
         0.68334967, 0.6807079 , 0.6617872 ],
        [0.3643731 , 0.5444552 , 0.34602585, 0.37885317, 0.5664864 ,
         0.68334967, 0.6807079 , 0.6617872 ],
        [0.3643731 , 0.5444552 , 0.34602585, 0.37885317, 0.5664864 ,
         0.68334967, 0.6807079 , 0.6617872 ],
        [0.3643731 , 0.5444552 , 0.34602585, 0.37885317, 0.5664864 ,
         0.68334967, 0.6807079 , 0.6617872 ],
        [0.3643731 , 0.5444552 , 0.34602585, 0.37885317, 0.5664864 ,
         0.68334967, 0.6807079 , 0.6617872 ],
        [0.3643731 

In [177]:
_expand(tf.tanh(M_t), dim=0, N=54)

<tf.Tensor: id=116613, shape=(54, 15, 8), dtype=float32, numpy=
array([[[0.5734388 , 0.7597851 , 0.623947  , ..., 0.7366561 ,
         0.52591425, 0.73811144],
        [0.70416963, 0.7093603 , 0.6100299 , ..., 0.5819585 ,
         0.7151722 , 0.6356709 ],
        [0.69485813, 0.5263842 , 0.71951   , ..., 0.7503891 ,
         0.64536816, 0.7427778 ],
        ...,
        [0.6784373 , 0.57028264, 0.72560006, ..., 0.5346641 ,
         0.6853796 , 0.64712423],
        [0.75006056, 0.6676223 , 0.66696376, ..., 0.63237935,
         0.66819984, 0.5410638 ],
        [0.6897298 , 0.67942226, 0.64429367, ..., 0.67885107,
         0.70504624, 0.62211007]],

       [[0.5734388 , 0.7597851 , 0.623947  , ..., 0.7366561 ,
         0.52591425, 0.73811144],
        [0.70416963, 0.7093603 , 0.6100299 , ..., 0.5819585 ,
         0.7151722 , 0.6356709 ],
        [0.69485813, 0.5263842 , 0.71951   , ..., 0.7503891 ,
         0.64536816, 0.7427778 ],
        ...,
        [0.6784373 , 0.57028264, 0.72560006,

In [178]:
N,M

(15, 8)

In [183]:
batch_size = 54
features = 20
inputs = tf.random.uniform((batch_size,features))
n_RH = 2
N = 120
M = 28

In [185]:
r_vector_list = tf.random.uniform((n_RH,batch_size,M))

In [188]:
r_vector_list

<tf.Tensor: id=116641, shape=(2, 54, 28), dtype=float32, numpy=
array([[[0.1414398 , 0.3842882 , 0.6772479 , ..., 0.969646  ,
         0.37744462, 0.9973525 ],
        [0.6472951 , 0.72290194, 0.21282566, ..., 0.28720188,
         0.19527411, 0.9012029 ],
        [0.6755738 , 0.9845519 , 0.398422  , ..., 0.6543113 ,
         0.38553083, 0.63730085],
        ...,
        [0.52165854, 0.16400516, 0.6842307 , ..., 0.17416847,
         0.6016805 , 0.9025004 ],
        [0.49156213, 0.9763869 , 0.63306665, ..., 0.626781  ,
         0.27169216, 0.82595813],
        [0.7387738 , 0.97463   , 0.1625737 , ..., 0.07111514,
         0.2527045 , 0.82427096]],

       [[0.7226045 , 0.33932137, 0.8060051 , ..., 0.25526655,
         0.27908754, 0.87092304],
        [0.34164107, 0.09027421, 0.08450985, ..., 0.6581018 ,
         0.37719667, 0.05956793],
        [0.8894453 , 0.50356495, 0.5786587 , ..., 0.43773568,
         0.01841354, 0.9084431 ],
        ...,
        [0.4358889 , 0.88747525, 0.5239773 ,

In [193]:
init_r = [tf.random.uniform([M]) for _ in range(n_RH)]

In [195]:
r_vector_list = [_expand(tf.nn.tanh(init_r[i]), dim=0, N=batch_size)
                                 for i in range(n_RH)]

In [197]:
r_vector_list

[<tf.Tensor: id=117017, shape=(54, 28), dtype=float32, numpy=
 array([[0.1497504 , 0.6949559 , 0.68602085, ..., 0.33641857, 0.13806678,
         0.7257407 ],
        [0.1497504 , 0.6949559 , 0.68602085, ..., 0.33641857, 0.13806678,
         0.7257407 ],
        [0.1497504 , 0.6949559 , 0.68602085, ..., 0.33641857, 0.13806678,
         0.7257407 ],
        ...,
        [0.1497504 , 0.6949559 , 0.68602085, ..., 0.33641857, 0.13806678,
         0.7257407 ],
        [0.1497504 , 0.6949559 , 0.68602085, ..., 0.33641857, 0.13806678,
         0.7257407 ],
        [0.1497504 , 0.6949559 , 0.68602085, ..., 0.33641857, 0.13806678,
         0.7257407 ]], dtype=float32)>,
 <tf.Tensor: id=117128, shape=(54, 28), dtype=float32, numpy=
 array([[0.70605516, 0.26320115, 0.3369606 , ..., 0.07776394, 0.48419595,
         0.4855323 ],
        [0.70605516, 0.26320115, 0.3369606 , ..., 0.07776394, 0.48419595,
         0.4855323 ],
        [0.70605516, 0.26320115, 0.3369606 , ..., 0.07776394, 0.48419595,
   

In [198]:
[inputs]

[<tf.Tensor: id=116627, shape=(54, 20), dtype=float32, numpy=
 array([[0.63553405, 0.82729447, 0.72089565, ..., 0.29759133, 0.02938223,
         0.46231174],
        [0.27687585, 0.65886426, 0.3074242 , ..., 0.73042274, 0.757058  ,
         0.17669225],
        [0.0813297 , 0.6152452 , 0.511917  , ..., 0.89991224, 0.65799344,
         0.72661006],
        ...,
        [0.50193846, 0.76605344, 0.24583673, ..., 0.99915147, 0.5454838 ,
         0.4732983 ],
        [0.4846015 , 0.77663994, 0.78144395, ..., 0.26790655, 0.81085575,
         0.8051983 ],
        [0.28491294, 0.45997107, 0.10576379, ..., 0.19729412, 0.5613487 ,
         0.02292502]], dtype=float32)>]

In [203]:
tf.concat([inputs] + r_vector_list, axis = 1)

<tf.Tensor: id=117136, shape=(54, 76), dtype=float32, numpy=
array([[0.63553405, 0.82729447, 0.72089565, ..., 0.07776394, 0.48419595,
        0.4855323 ],
       [0.27687585, 0.65886426, 0.3074242 , ..., 0.07776394, 0.48419595,
        0.4855323 ],
       [0.0813297 , 0.6152452 , 0.511917  , ..., 0.07776394, 0.48419595,
        0.4855323 ],
       ...,
       [0.50193846, 0.76605344, 0.24583673, ..., 0.07776394, 0.48419595,
        0.4855323 ],
       [0.4846015 , 0.77663994, 0.78144395, ..., 0.07776394, 0.48419595,
        0.4855323 ],
       [0.28491294, 0.45997107, 0.10576379, ..., 0.07776394, 0.48419595,
        0.4855323 ]], dtype=float32)>